In [3]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.7 environment at: C:\Azhar\FullStackRAG\.venv
Audited 6 packages in 37.73s


In [4]:
! uv pip install langchain-google-genai

Using Python 3.12.7 environment at: C:\Azhar\FullStackRAG\.venv
Resolved 39 packages in 1.45s
Prepared 13 packages in 1.72s
Installed 13 packages in 639ms
 + cachetools==6.2.0
 + filetype==1.2.0
 + google-ai-generativelanguage==0.7.0
 + google-api-core==2.25.2
 + google-auth==2.41.1
 + googleapis-common-protos==1.70.0
 + grpcio==1.75.1
 + grpcio-status==1.75.1
 + langchain-google-genai==2.1.12
 + proto-plus==1.26.1
 + pyasn1==0.6.1
 + pyasn1-modules==0.4.2
 + rsa==4.9.1


# Data Ingestion

In [5]:
from langchain.document_loaders import TextLoader

In [9]:
loader = TextLoader("C:/Azhar/FullStackRAG/data/agenticAI.txt", encoding="utf8")
documents = loader.load()

In [10]:
documents[0].page_content[:500]  # Print the first 500 characters of the first documen

'Agentic AI refers to autonomous, goal-oriented systems that can independently make decisions, plan, and execute multi-step tasks with minimal human intervention. Unlike traditional, rule-based AI or reactive generative AI, an agentic AI system is proactive and adaptive, continuously perceiving its environment, learning from outcomes, and adjusting its behavior to achieve complex objectives. \nHow agentic AI works\nAgentic AI operates through an iterative, four-stage process that allows it to funct'

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [13]:
text_chunks=text_splitter.split_documents(documents)
text_chunks

[Document(metadata={'source': 'C:/Azhar/FullStackRAG/data/agenticAI.txt'}, page_content='Agentic AI refers to autonomous, goal-oriented systems that can independently make decisions, plan, and execute multi-step tasks with minimal human intervention. Unlike traditional, rule-based AI or'),
 Document(metadata={'source': 'C:/Azhar/FullStackRAG/data/agenticAI.txt'}, page_content='rule-based AI or reactive generative AI, an agentic AI system is proactive and adaptive, continuously perceiving its environment, learning from outcomes, and adjusting its behavior to achieve complex'),
 Document(metadata={'source': 'C:/Azhar/FullStackRAG/data/agenticAI.txt'}, page_content='to achieve complex objectives.'),
 Document(metadata={'source': 'C:/Azhar/FullStackRAG/data/agenticAI.txt'}, page_content='How agentic AI works\nAgentic AI operates through an iterative, four-stage process that allows it to function autonomously:'),
 Document(metadata={'source': 'C:/Azhar/FullStackRAG/data/agenticAI.txt'}, pag

In [14]:
! uv pip install faiss-cpu

Using Python 3.12.7 environment at: C:\Azhar\FullStackRAG\.venv
Resolved 3 packages in 291ms
Prepared 1 package in 5.58s
Installed 1 package in 78ms
 + faiss-cpu==1.12.0


In [15]:
from langchain.vectorstores import FAISS

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\azhar\AppData\Local\Temp\ipykernel_10884\1822540165.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Azhar\FullStackRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)
vectorstore

In [18]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

Document 1:
Agentic AI refers to autonomous, goal-oriented systems that can independently make decisions, plan, and execute multi-step tasks with minimal human intervention. Unlike traditional, rule-based AI or
--------------------------------------------------
Document 2:
How agentic AI works
Agentic AI operates through an iterative, four-stage process that allows it to function autonomously:
--------------------------------------------------
Document 3:
Core components
A complete agentic AI architecture typically includes several key components:
--------------------------------------------------
Document 4:
AI Agents: Individual, specialized AI components that can work independently or collaboratively within a larger system.
--------------------------------------------------


In [19]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [20]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [21]:
from langchain.schema.output_parser import StrOutputParser

In [22]:
output_parser=StrOutputParser()

In [ ]:
import os
from langchain_google_genai import GoogleGenerativeAI
os.environ['Gemini_API_KEY']=""
os.environ["GOOGLE_API_KEY"]="" 

In [26]:
llm_model = GoogleGenerativeAI(model="gemini-2.5-flash")

In [25]:
retriever=vectorstore.as_retriever()

In [27]:
from langchain.schema.runnable import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [28]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI refers to autonomous, goal-oriented systems capable of independently making decisions, planning, and executing multi-step tasks with minimal human intervention. Unlike traditional, rule-based AI, it functions autonomously through an iterative, four-stage process. A complete agentic AI architecture typically includes several key components. Among these are AI Agents, which are individual, specialized AI components that can work independently or collaboratively within a larger system.'